Datafun-07-ml
Introduction to Machine Learning
Author: Brenda Fuemmeler
Date: 2-23-25
https://github.com/bfuemmeler/datafun-07-ml

In [2]:
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import pyarrow as pa
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy.stats import linregress
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression